## 2) Comparing Models - Prudential Life

## Importing Packages and loading into DF

In [3]:
from __future__ import print_function
import timeit

from sklearn.model_selection import learning_curve, ShuffleSplit
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

%matplotlib inline

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgboost
from xgboost import XGBClassifier


from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV

import sklearn.ensemble
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn import metrics
from sklearn.metrics import log_loss, confusion_matrix, mean_squared_error, accuracy_score

import lime
import lime.lime_tabular

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
ins_data = './dataset/train.csv'
ins = pd.read_csv(ins_data)
# ins_test_data = './dataset/test.csv'
# ins_test = pd.read_csv(ins_test_data)

In [5]:
print(ins.shape)
# print(ins_test.shape)

(59381, 128)


## Cleaing Data
- Dropping Axis Column
- Creating dummy for column Product_Info_2 column
- Removing NaN values - where normally distributed, replacing with mean value, where other columns used meadian.

In [6]:
ins.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.08,2,1,1,0.64,0.58,0.15,0.32,0.03,12,1,0.00,3,nan,1,2,6,3,1,2,1,1,1,3,1,0.00,1,1,2,2,nan,0.60,nan,0.53,4.00,112,2,1,1,3,2,2,1,nan,3,2,3,3,240.00,3,3,1,1,2,1,2,3,nan,1,3,3,1,3,2,3,nan,1,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.08,2,3,1,0.06,0.60,0.13,0.27,0.00,1,3,0.00,2,0.00,1,2,6,3,1,2,1,2,1,3,1,0.00,1,3,2,2,0.19,nan,0.08,nan,5.00,412,2,1,1,3,2,2,1,nan,3,2,3,3,0.00,1,3,1,1,2,1,2,3,nan,1,3,3,1,3,2,3,nan,3,1,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.08,2,3,1,0.03,0.75,0.29,0.43,0.03,9,1,0.00,2,0.03,1,2,8,3,1,1,1,2,1,1,3,nan,3,2,3,3,0.30,nan,0.23,nan,10.00,3,2,2,1,3,2,2,2,nan,3,2,3,3,nan,1,3,1,1,2,1,2,3,nan,2,2,3,1,3,2,3,nan,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.49,2,3,1,0.16,0.67,0.21,0.35,0.04,9,1,0.00,3,0.20,2,2,8,3,1,2,1,2,1,1,3,nan,3,2,3,3,0.42,nan,0.35,nan,0.00,350,2,2,1,3,2,2,2,nan,3,2,3,3,nan,1,3,1,1,2,2,2,3,nan,1,3,3,1,3,2,3,nan,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.23,2,3,1,0.42,0.65,0.23,0.42,0.03,9,1,0.00,2,0.05,1,2,6,3,1,2,1,2,1,1,3,nan,3,2,3,2,0.46,nan,0.41,nan,nan,162,2,2,1,3,2,2,2,nan,3,2,3,3,nan,1,3,1,1,2,1,2,3,nan,2,2,3,1,3,2,3,nan,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8


In [7]:
ins.describe()

,Id,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
count,"59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,362.00","59,381.00","59,381.00","52,602.00","59,381.00","48,527.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","33,985.00","59,381.00","59,381.00","59,381.00","59,381.00","30,725.00","25,140.00","40,197.00","17,570.00","50,492.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00",557.00,"59,381.00","59,381.00","59,381.00","59,381.00","14,785.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","3,801.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","1,107.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00","59,381.00"
mean,"39,507.21",1.03,24.42,0.33,2.01,2.67,1.04,0.41,0.71,0.29,0.47,0.08,8.64,1.30,0.01,2.14,0.36,1.21,2.01,5.84,2.88,1.03,1.41,1.04,1.73,1.06,2.15,1.96,0.00,1.90,2.05,2.42,2.69,0.47,0.50,0.44,0.48,7.96,253.99,2.10,1.65,1.01,2.89,2.01,2.04,1.77,14

In [9]:
ins.drop('Id',axis=1,inplace=True)
# ins_test.drop('Id',axis=1,inplace=True)

In [10]:
ins.Product_Info_2.value_counts()
prod_2 = {'A1':1,'A2':2,'A3':3,'A4':4,'A5':5,'A6':6,'A7':7,'A8':8,'B1':9,'B2':10,'C1':11,'C2':12,
          'C3':13,'C4':14,'D1':15,'D2':16,'D3':17,'D4':18,'E1':19}

ins.replace({"Product_Info_2": prod_2},inplace=True)

In [11]:
for i in range(2,6):
    ins['Family_Hist_%i' % i].fillna(0,inplace=True)
#     ins_test['Family_Hist_%i' % i].fillna(0,inplace=True)

for i in [1,10,15,24,32]:
    ins['Medical_History_%i' % i].fillna(0,inplace=True)
#     ins_test['Medical_History_%i' % i].fillna(0,inplace=True)

for i in [1,4,6]:
    ins['Employment_Info_%i' % i].fillna(0,inplace=True)
#     ins_test['Employment_Info_%i' % i].fillna(0,inplace=True)

ins['Insurance_History_5'].fillna(0,inplace=True)
# ins_test['Insurance_History_5'].fillna(0,inplace=True)

## Creating X and y for modelling

In [12]:
for i in ins.columns:
    if str(ins[i].dtypes) == 'int64':
        ins[i] = ins[i].astype('int8')
    else:
        ins[i] = ins[i].astype('float16')

In [13]:
features = ins.drop(['Response','Ht','Wt'],axis=1)
targets = ins.Response
targets = targets.map(lambda x: x-1)
X, X_holdout, y, y_holdout = train_test_split(features,targets, train_size = 0.8,test_size=0.2,random_state=77)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.9,test_size=0.1,random_state=50)

#### Determining Baseline

In [14]:
y_train.value_counts()

7    14028
5     8069
6     5821
1     4714
0     4489
4     3906
3     1003
2      723
Name: Response, dtype: int64

In [31]:
baseline=15637/len(y_train)
print(baseline)

0.36575211096297333


In [34]:
print('Baseline: 0.37')

Baseline: 0.37


In [23]:
## general purpose function to test models.
def fit_model(alg):
    alg.fit(X_train,y_train)
    scores= cross_val_score(alg,X_train,y_train,cv=5,n_jobs=-1,verbose=1)
    ## calculate accuracy
    accuracy_score_test = alg.score(X_test,y_test)
    accuracy_score_train = alg.score(X_train,y_train)
    ## calculate log loss
    y_pred = alg.predict_proba(X_train)
    log_loss_train = log_loss(y_train,y_pred)
    y_pred = alg.predict_proba(X_test)
    log_loss_test = log_loss(y_test,y_pred)
    ## print values
    print('baseline: ',round(baseline,2))
    print("mean cv scores: ",round(np.mean(scores),2))
    print("std cv scores: ",round(np.std(scores),2))
    print('-------------------------------')
    print("accuracy score - train: ",round(accuracy_score_train,2))
    print("accuracy score - test: ",round(accuracy_score_test,2))
    print('-------------------------------')
    print("log loss score - train: ",round(log_loss_train,2))
    print("log loss score - test: ",round(log_loss_test,2))

## KNN

In [24]:
knn = KNeighborsClassifier(n_neighbors=3,leaf_size=30,n_jobs=-1)
fit_model(knn)

C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    8.9s remaining:   13.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.0s finished
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: 

baseline:  0.37
mean cv scores:  0.27
std cv scores:  0.0
-------------------------------
accuracy score - train:  0.59
accuracy score - test:  0.27
-------------------------------
log loss score - train:  0.76
log loss score - test:  16.51


Various results

CV - results (n_neighbors=4,leaf_size=30,n_jobs=3)

[0.31529109 0.31579563 0.31750672 0.31278512 0.31675638]
0.31562698836623093

CV - results (n_neighbors=5)

([0.29261164 0.29276277 0.30218688 0.29687683 0.29581091]
0.29604980581700885

## Logistic Regression

In [25]:
lr = LogisticRegression(multi_class='auto', solver = 'liblinear',max_iter=200)
fit_model(lr)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


baseline:  0.37
mean cv scores:  0.49
std cv scores:  0.0
-------------------------------
accuracy score - train:  0.5
accuracy score - test:  0.5
-------------------------------
log loss score - train:  1.37
log loss score - test:  1.4


CV Results - (multi_class='auto', solver = 'liblinear')

[0.49590835 0.49397872 0.49923985 0.49584747 0.49075591]
0.4951460599227767

In [19]:
y_train.value_counts(normalize=True)

7   0.33
5   0.19
6   0.14
1   0.11
0   0.10
4   0.09
3   0.02
2   0.02
Name: Response, dtype: float64

## SVM

In [ ]:
parameters = {'C':[0.001,0.25,0.5,0.75,1,10],
              'kernel':['linear','rbf'],
              'gamma':['scale',0.001,0.01,0.1,1,2,3],
              'decision_function_shape':['ovo','ovr']}

In [ ]:
svm = SVC(probability=True,random_state=42)
grid_search = GridSearchCV(svm,param_grid=parameters,cv=3,n_jobs=3)
grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_params_

In [26]:
svm = SVC(C=1, kernel='rbf', gamma='scale',decision_function_shape='ovr',probability=True,random_state=42)
fit_model(svm)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 51.6min remaining: 77.4min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 51.8min finished


baseline:  0.37
mean cv scores:  0.35
std cv scores:  0.0
-------------------------------
accuracy score - train:  0.35
accuracy score - test:  0.35
-------------------------------
log loss score - train:  1.61
log loss score - test:  1.62


In [19]:
scores = cross_val_score(svm, X_train, y_train, cv=5,n_jobs=-1,verbose=1)
predictions = cross_val_predict(svm, X_train,y_train,cv=5,n_jobs=3)
r2 = metrics.r2_score(y_train, predictions)
print(scores)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 64.1min remaining: 96.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 64.3min finished


[0.36029928 0.36244593 0.3573851  0.35466136 0.35490288]
0.35793890883976404


CV Scores - (cv=5,n_jobs=-1,verbose=1)

[0.36029928 0.36244593 0.3573851  0.35466136 0.35490288]
0.35793890883976404

In [ ]:
svm.fit(X_train,y_train)
accuracy_score_train = grid_search.best_score_

accuracy_score_test = grid_search.score(X_test,y_test)

y_pred = grid_search.predict_proba(X_test)
log_loss_test = log_loss(y_test,y_pred)

y_pred = grid_search.predict_proba(X_train)
log_loss_train = log_loss(y_train,y_pred)

print('baseline: ',baseline)
print('-------------------------------')
print("accuracy score - train: ",accuracy_score_train)
print("accuracy score - test: ",accuracy_score_test)
print('-------------------------------')
print("log loss score - train: ",log_loss_train)
print("log loss score - test: ",log_loss_test)

Test 1
- C=0.001, kernel='rbf', gamma='scale',decision_function_shape='ovr',probability=True,random_state=42

## Random Forest

In [27]:
forest = RandomForestClassifier(max_depth=5,max_leaf_nodes=30,verbose=1)
fit_model(forest)

C:\Users\Roy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   14.6s remaining:   22.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


baseline:  0.37
mean cv scores:  0.44
std cv scores:  0.01
-------------------------------
accuracy score - train:  0.45
accuracy score - test:  0.45
-------------------------------
log loss score - train:  1.52
log loss score - test:  1.53


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


CV Results (max_depth=5,max_leaf_nodes=30,class_weight='balanced',verbose=1)

[0.42646715 0.41190226 0.42673372 0.40905369 0.41048444]
0.4169282498796433

CV Results (max_depth=5,max_leaf_nodes=30,verbose=1)

[0.44201543 0.44557465 0.46099871 0.43853082 0.4678212 ]
0.4509881644734108

In [27]:
forest_features = pd.DataFrame()
forest_features['feature'] = X.columns
forest_features['importance'] = forest.feature_importances_
forest_features.sort_values('importance',ascending=False).head(10)

,feature,importance
8,BMI,0.260038
90,Medical_Keyword_15,0.192172
57,Medical_History_23,0.100389
38,Medical_History_4,0.098857
3,Product_Info_4,0.090573
7,Ins_Age,0.047791
49,Medical_History_15,0.040456
20,InsuredInfo_6,0.022139
78,Medical_Keyword_3,0.020943
64,Medical_History_30,0.012430


## XGBoost

In [28]:
from xgboost import XGBClassifier
model = XGBClassifier(learning_rate=0.01,max_depth= 7,eta= 0.1,verbosity= 1,objective= 'multi:softprob',num_class= 7,reg_lambda=0)
eval_set = [(X_train, y_train), (X_test, y_test)]
model.fit(X_train,y_train,eval_metric=['merror','mlogloss'],eval_set=eval_set, verbose=True)
fit_model(model)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  8.7min remaining: 13.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  8.8min finished


baseline:  0.37
mean cv scores:  0.56
std cv scores:  0.0
-------------------------------
accuracy score - train:  0.6
accuracy score - test:  0.56
-------------------------------
log loss score - train:  1.46
log loss score - test:  1.51


In [38]:
model = XGBClassifier(n_estimators=90,max_depth=6,min_child_weight=5,eta=0.01,verbosity= 1,colsample_bytree=0.5,objective= 'multi:softprob',num_class= 8)
scores = cross_val_score(model, X_train, y_train, cv=5,n_jobs=-1,verbose=1)
predictions = cross_val_predict(model, X_train,y_train,cv=5,n_jobs=3)
r2 = metrics.r2_score(y_train, predictions)
print(scores)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.5min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.6min finished


[0.5748188  0.57792587 0.57572214 0.56883846 0.57102738]
0.5736665303633456


CV Results - XGBClassifier(max_depth= 7,eta= 0.1,verbosity= 1,objective= 'multi:softprob',num_class=8,alpha=0.8,reg_lambda=50)

[0.57657236 0.57780896 0.57595603 0.56743479 0.56938919]
0.5734322647763243

CV Results - n_estimators=90,max_depth=5,eta=0.1,verbosity= 1,colsample_bytree=0.5,objective= 'multi:softprob',num_class= 8,reg_lambda=0

[0.57610475 0.57359991 0.57595603 0.56392561 0.57126141]
0.572169539099591

n_estimators=90,max_depth=5,eta=0.01,verbosity= 1,colsample_bytree=0.5,objective= 'multi:softprob',num_class= 8,reg_lambda=0)

[0.57610475 0.57359991 0.57595603 0.56392561 0.57126141]
0.572169539099591

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

### XGBoost - Grid Search for Best Parameters

XGBoost appeared to perform the best out of all my models. As such, the foucus will be on this model going forward. The following section attempts to improve the performance of the XGBoost model through the use of gridsearch.

In [35]:
parameters = {'max_depth':[4,5,6,7,8,9],
              'lerning_rate':[0.01,0.05,0.1,1],
              'gamma':[0,1],
              'min_child_weight':[1,5,10,50,100]}

In [36]:
model = XGBClassifier(n_estimators=90,colsample_bytree=0.5,verbosity= 1,objective= 'multi:softprob',num_class= 8)
grid_search = GridSearchCV(model,param_grid=parameters,cv=5,n_jobs=-1)
grid_search.fit(X_test,y_test)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=90,
       n_jobs=1, nthread=None, num_class=8, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [4, 5, 6, 7, 8, 9], 'lerning_rate': [0.01, 0.05, 0.1, 1], 'gamma': [0, 1], 'min_child_weight': [1, 5, 10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [37]:
grid_search.best_params_

{'gamma': 1, 'lerning_rate': 0.01, 'max_depth': 6, 'min_child_weight': 5}

### XGBoost - Further tuning to reduce overfitting

While the model with the above grid search parameters performed well, it was overfitting to the training data. As such, below we include some regularisation parameters to reduce the overfitting problem, arriving at our final results.

In [29]:
xgb = XGBClassifier(learning_rate = 0.01,n_estimators = 100,max_depth = 6,subsample = 0.8,
                    min_child_weight = 5,eta=0.02,verbosity = 1,
                    colsample_bytree = 0.6,objective= 'multi:softprob',num_class = 8,
                    gamma = 5, reg_lambda = 10, reg_alpha = 6)

fit_model(xgb)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.3min remaining:  7.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.4min finished


baseline:  0.37
mean cv scores:  0.57
std cv scores:  0.0
-------------------------------
accuracy score - train:  0.58
accuracy score - test:  0.57
-------------------------------
log loss score - train:  1.19
log loss score - test:  1.22


Plese note that I have a seperate notebook that goes through a more manual and rigorous approach in tweeking the prameters for XGBoost. The results were fairly similar to the above although the hyperparameters were somewhat different.

In [41]:
## for lime code only due to column heading requirements.
xgb.fit(X_train.values,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.6, eta=0.02, gamma=5,
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, num_class=8, objective='multi:softprob',
       random_state=0, reg_alpha=6, reg_lambda=10, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.8, verbosity=1)

## Extra Trees

In [30]:
extra_tree = ExtraTreesClassifier(max_depth=8,max_features=11,
                                 bootstrap=True,n_jobs=-1,
                                 random_state=1,verbose=1)
fit_model(extra_tree)

C:\Users\Roy\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.4s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


baseline:  0.37
mean cv scores:  0.43
std cv scores:  0.01
-------------------------------
accuracy score - train:  0.46
accuracy score - test:  0.45
-------------------------------
log loss score - train:  1.53
log loss score - test:  1.55


## SMOTE 
-Smote was used and then new data was tested on the XGB Classifier. This did not  seem to improve results.

In [24]:
smt=SMOTE(k_neighbors=5)
X2_train,y2_train = smt.fit_sample(X_train,y_train)
np.bincount(y2_train)

array([    0, 14053, 14053, 14053, 14053, 14053, 14053, 14053, 14053],
      dtype=int64)

In [25]:
model2 = XGBClassifier(n_estimators=100,max_depth=5,eta=0.01,verbosity= 1,colsample_bytree=0.7,objective= 'multi:softprob',num_class= 8,reg_lambda=0,n_jobs=-1)
scores = cross_val_score(model2, X_train, y_train, cv=5,n_jobs=3,verbose=1)
predictions = cross_val_predict(model2, X_train,y_train,cv=5,n_jobs=3)
r2 = metrics.r2_score(y_train, predictions)
print(scores)
print(scores.mean())

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:  4.2min finished


[0.57598784 0.57476909 0.5753713  0.56614809 0.5711444 ]
0.5726841425903546


## RFE Attempt

A seperate notebook explores RFE more. I attempted to look at the top 5 and the top 20 features only, fitting them to the above models. Some models performed better (KNN and Random Tree) while others, including our best performer (XGBoost) performed significantly worse. Refer to other notebook for more details.

Interesting note here is that RFE rated features were quite consistent with outputs from SHAP (see below) in terms of the most impactful features.

In [31]:
from sklearn.feature_selection import RFE
estimator = XGBClassifier(n_estimators=100,max_depth=5,eta=0.01,verbosity= 1,colsample_bytree=0.7,objective= 'multi:softprob',num_class= 8,reg_lambda=0,n_jobs=-1)
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X,y)

print(selector.supports_)
print(selector.ranking_)


KeyboardInterrupt: 

## Learning Curves

In [ ]:
from sklearn.model_selection import learning_curve, ShuffleSplit

In [24]:
def plot_learning_curve(estimator, title, X, y, ylim=None,cv=None,n_jobs=3,train_sizes=np.linspace(.1,1.0,5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training Samples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1bbg)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score")

    plt.legend(loc="best")
    return plt

In [ ]:
title = "Learning Curve (XGBoost)"
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
estimator = model
plot_learning_curve(estimator, title, X_train, y_train, ylim=(0.0, 1.00), cv=cv, n_jobs=3)

## Confusion Matrix

Creating some confusion matrix to see how my modesl are predicting. In general the model seems to err on the "aggressive" side and predics higher classes (towards 8). This was true of all the models.

The tree models also did not predict any class 3 or 4 which would have been an issue if we wanted to use those models.

In [53]:
y_hat = model.predict(X_test)
conmat = np.array(confusion_matrix(y_test,y_hat, labels=[0,1,2,3,4,5,6,7]))
confusion = pd.DataFrame(conmat, index=['is_1', 'is_2', 'is_3', 'is_4', 'is_5', 'is_6', 'is_7', 'is_8'],
                         columns=['predicted_1','predicted_2','predicted_3','predicted_4','predicted_5','predicted_6','predicted_7','predicted_8'])

In [54]:
confusion

,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8
is_1,73,54,5,15,36,129,63,126
is_2,32,100,6,6,51,105,55,130
is_3,1,3,16,16,15,15,0,6
is_4,2,0,2,58,0,37,3,17
is_5,13,33,4,0,223,93,35,53
is_6,23,25,1,5,34,473,113,237
is_7,0,4,1,0,2,108,234,271
is_8,3,1,0,1,6,59,43,1476


In [55]:
y_hat = lr.predict(X_test.values)
conmat = np.array(confusion_matrix(y_test,y_hat, labels=[0,1,2,3,4,5,6,7]))
confusion = pd.DataFrame(conmat, index=['is_1', 'is_2', 'is_3', 'is_4', 'is_5', 'is_6', 'is_7', 'is_8'],
                         columns=['predicted_1','predicted_2','predicted_3','predicted_4','predicted_5','predicted_6','predicted_7','predicted_8'])
confusion

C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8
is_1,125,57,1,9,31,104,58,116
is_2,69,77,2,5,59,114,38,121
is_3,2,4,8,4,5,42,0,7
is_4,3,0,0,29,0,67,2,18
is_5,31,38,1,2,105,104,56,117
is_6,67,49,1,3,35,380,98,278
is_7,18,14,0,0,6,96,156,330
is_8,8,5,1,0,8,73,19,1475


In [56]:
y_hat = forest.predict(X_test.values)
conmat = np.array(confusion_matrix(y_test,y_hat, labels=[0,1,2,3,4,5,6,7]))
confusion = pd.DataFrame(conmat, index=['is_1', 'is_2', 'is_3', 'is_4', 'is_5', 'is_6', 'is_7', 'is_8'],
                         columns=['predicted_1','predicted_2','predicted_3','predicted_4','predicted_5','predicted_6','predicted_7','predicted_8'])
confusion

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8
is_1,43,45,0,0,21,162,22,208
is_2,24,44,0,0,24,145,18,230
is_3,1,1,0,0,7,39,0,24
is_4,1,0,0,0,0,41,0,77
is_5,7,22,0,0,108,134,9,174
is_6,16,18,0,0,0,361,23,493
is_7,4,2,0,0,0,119,46,449
is_8,0,2,0,0,0,48,1,1538


In [57]:
y_hat = extra_tree.predict(X_test.values)
conmat = np.array(confusion_matrix(y_test,y_hat, labels=[0,1,2,3,4,5,6,7]))
confusion = pd.DataFrame(conmat, index=['is_1', 'is_2', 'is_3', 'is_4', 'is_5', 'is_6', 'is_7', 'is_8'],
                         columns=['predicted_1','predicted_2','predicted_3','predicted_4','predicted_5','predicted_6','predicted_7','predicted_8'])
confusion

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished


,predicted_1,predicted_2,predicted_3,predicted_4,predicted_5,predicted_6,predicted_7,predicted_8
is_1,93,45,0,0,0,121,52,190
is_2,40,48,0,0,2,131,45,219
is_3,1,0,0,0,0,42,3,26
is_4,2,0,0,0,0,56,1,60
is_5,16,16,0,0,0,116,52,254
is_6,32,18,0,0,1,388,89,383
is_7,8,4,0,0,0,97,124,387
is_8,4,1,0,0,0,110,10,1464


## PICKLE

In [36]:
import dill

In [133]:
pickle_file_path = './xgboost_ins.pkl'

In [134]:
def serialize(model):
    with open(pickle_file_path, 'wb') as f:
        dill.dump(model, f)

In [37]:
serialize(model)

NameError: name 'serialize' is not defined

In [38]:
pickle_file_path = './lime_ins.pkl'

In [39]:
def serialize(explainer):
    with open(pickle_file_path, 'wb') as f:
        dill.dump(explainer, f)

In [40]:
serialize(explainer)

## Ensemble - MLENS GBC

One more attempt at improving accuracy / log loss scores. The Ensemble of 4 model was used where the meta learner was a Gradient Boosting Classifier. While performance was good, it was still not better than XGBoost and considering this requires 4 other models, this is disregarded.

In [47]:
from mlens.ensemble import SuperLearner

ensemble = SuperLearner(scorer=log_loss,random_state=42,verbose=2)
ensemble.add([lr,forest,extra_tree,xgb])
ensemble.add_meta(GradientBoostingClassifier(n_estimators=1000,max_depth=5,subsample=0.8,learning_rate=0.001,random_state=42))

TypeError: '<module 'xgboost' from 'D:\\Anaconda\\lib\\site-packages\\xgboost\\__init__.py'>' does not appear to be a valid estimator as it does not implement a 'get_params' method. Type: <class 'module'>

In [39]:
ensemble.fit(X_train,y_train)


Fitting 2 layers
Processing layer-1             

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.

done | 00:01:47
Processing layer-2             done | 00:05:27
Fit complete                        | 00:07:15


D:\Anaconda\lib\site-packages\mlens\parallel\_base_functions.py:226: MetricWarning: [gradientboostingclassifier.0.0] Could not score gradientboostingclassifier. Details:
ValueError('y_true and y_pred contain different number of classes 8, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4 5 6 7]')
  (name, inst_name, exc), MetricWarning)


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=7270, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...7D0D9D8>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=42, sample_size=20,
       scorer=<function log_loss at 0x0000000017D0D9D8>, shuffle=False,
       verbose=2)

In [40]:
preds=ensemble.predict(X_train)


Predicting 2 layers
Processing layer-1             

C:\Users\rl\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
D:\Anaconda\lib\site-packages\sklearn\ensemble\base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


done | 00:00:00
Processing layer-2             done | 00:00:12
Predict complete                    | 00:00:13


In [41]:
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                    ft-m  ft-s  pt-m  pt-s
layer-1  extratreesclassifier       0.57  0.03  0.78  0.01
layer-1  logisticregression        60.01  1.13  0.06  0.01
layer-1  randomforestclassifier     0.54  0.20  0.11  0.02



In [45]:
print("Prediction score: %.3f" % accuracy_score(preds, y_train))

Prediction score: 0.522


## Naive Bayes

Last model that was not previously tried. Unfortunately, this did not seem to perform well.

In [42]:
from sklearn import naive_bayes

In [60]:
bayes=naive_bayes.MultinomialNB(alpha=1)

In [64]:
cross_val_score(bayes,X_train.drop('Medical_History_2',axis=1),y_train,cv=2)

C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
C:\Users\Roy\Anaconda3\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


array([0.05117171, 0.05371012])

In [58]:
print("accuracy score - train: 0.58")
print("accuracy score - test: 0.57")
print("----------------------------")
print("log loss score - train: 1.19")
print("log loss score - test: 1.22")

accuracy score - train: 0.58
accuracy score - test: 0.57
----------------------------
log loss score - train: 1.19
log loss score - test: 1.22


In [58]:
X_train.head()

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
38017,1,15,26,0.281982,2,1,1,0.746094,0.487061,0.048004,9,1,0.0,2,0.000000,2,2,6,3,1,1,1,1,1,3,1,0.002001,1,1,2,3,0.000000,0.480469,0.000000,0.687500,3.0,112,2,2,1,3,2,2,2,0.0,3,2,3,3,0.0,1,3,1,1,2,1,2,3,0.0,1,3,3,1,3,2,3,0.0,3,3,1,2,2,1,1,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
31191,1,6,26,0.076904,2,1,1,0.283691,0.487549,0.040985,9,1,0.0,2,0.099976,1,2,3,3,1,1,1,2,1,1,3,0.000000,3,2,3,2,0.463867,0.000000,0.323975,0.000000,3.0,-95,2,2,1,3,2,2,2,0.0,3,2,3,3,0.0,1,3,1,1,2,1,2,3,0.0,1,3,3,1,1,2,3,0.0,3,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3421,1,15,29,0.230713,2,3,1,0.746094,0.770996,0.431885,1,3,0.0,3,1.000000,1,2,3,3,1,1,1,2,1,3,1,0.001667,1,3,2,3,0.000000,0.000000,0.000000,0.490967,0.0,-21,2,1,1,3,1,2,2,0.0,3,2,3,3,191.0,3,3,2,1,2,1,2,1,0.0,1,3,3,1,1,3,3,0.0,3,1,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
42682,1,18,10,0.487061,2,3,1,0.193970,0.368896,0.059998,9,1,0.0,3,0.000000,1,2,3,3,1,2,1,1,1,3,1,0.001333,1,1,2,3,0.463867,0.000000,0.408447,0.000000,0.0,-94,2,2,1,3,2,2,2,0.0,3,2,3,3,0.0,1,3,1,1,2,1,2,3,0.0,1,3,3,1,3,2,3,0.0,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18613,1,8,26,0.035889,2,3,1,0.626953,0.447754,0.035004,9,1,0.0,2,0.000000,1,2,6,3,1,2,1,1,1,3,1,0.000467,1,1,2,3,0.000000,0.450928,0.732422,0.000000,5.0,-21,2,1,1,3,2,2,1,0.0,3,2,3,3,240.0,3,3,1,1,2,1,2,3,0.0,2,2,3,1,3,2,3,0.0,3,3,1,3,2,1,3,3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
